## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [48]:
import pandas as pd #импортирую иблиотеку pandas
data = pd.read_csv('/datasets/data.csv') #открываю файл с данными
print(data.head(10)) #печатаю первые 10 строк таблицы
print('_________________________________________________')
data.info() #получаю общую информацию о таблице
print('_________________________________________________')
#проверю уникальные значения в интересующих меня стобцах ниже
print('Уникальные значения в столбце "количество детей в семье": ', data['children'].unique())
print('Уникальные значения в столбце "семейное положение": ', data['family_status'].unique())
print('Уникальные значения в столбце "образование": ', data['education'].unique())
print('Уникальные значения в столбце "тип занятости": ', data['income_type'].unique())
print('Уникальные значения в столбце "цель получения кредита": ', data['purpose'].unique())
print('Уникальные значения в столбце "имел ли задолженность по возврату кредитов": ', data['debt'].unique())
print('_________________________________________________')
print('Проверка артефактов в столбце "количество детей в семье":')
print(data['children'].value_counts())

   children  days_employed  dob_years education  education_id  \
0         1   -8437.673028         42    высшее             0   
1         1   -4024.803754         36   среднее             1   
2         0   -5623.422610         33   Среднее             1   
3         3   -4124.747207         32   среднее             1   
4         0  340266.072047         53   среднее             1   
5         0    -926.185831         27    высшее             0   
6         0   -2879.202052         43    высшее             0   
7         0    -152.779569         50   СРЕДНЕЕ             1   
8         2   -6929.865299         35    ВЫСШЕЕ             0   
9         0   -2188.756445         41   среднее             1   

      family_status  family_status_id gender income_type  debt   total_income  \
0   женат / замужем                 0      F   сотрудник     0  253875.639453   
1   женат / замужем                 0      F   сотрудник     0  112080.014102   
2   женат / замужем                 0    

### Вывод

Рассмотрим полученную информацию подробнее.
Всего в таблице 12 столбцов и 21525 строк, типы данных в таблице: float64, int64, object..
Подробно разберём, какие в df столбцы и какую информацию они содержат:

##### children — количество детей в семье
##### days_employed — общий трудовой стаж в днях
##### dob_years — возраст клиента в годах
##### education — уровень образования клиента
##### education_id — идентификатор уровня образования
##### family_status — семейное положение
##### family_status_id — идентификатор семейного положения
##### gender — пол клиента
##### income_type — тип занятости
##### debt — имел ли задолженность по возврату кредитов
##### total_income — ежемесячный доход
##### purpose — цель получения кредита

Количество значений в столбцах различается. Это говорит о том, что в данных есть пропущенные значения.
Количество пропущенных значений в столбцах 'days_employed'и 'total_income' одинаковое, в связи с этим возникла гипотеза о том, что у категории безработных отсутствует доход. Эту гипотезу проверим в следующем шаге
В столбце 'days_employed' предоставлены данные c артефактами поскольку они имеют отрицательные и неправдободобные значения. Причина их возниконвения, вероятнее всего в преобразовании данных из одного типа в другой.
В столбце 'education' одни и те же данные об образовании заемщика написаны разным регистром. Необходимо устранить.
В столбце количество детей также имеются артефакты: '-1' ребенок и '20' детей. Показатель в 20 детей явный артефакт, поскольку мне удалось найти информацию только про одну семью в России с 20 детьми. Причина возникновения артефактов в этом столбце - человеческий фактор, а именно неверный ввод данных при заполнении анкет.

### Шаг 2. Предобработка данных

Обработка пропусков

In [49]:
# проверяю гипотезу из прошлого вывода о том, что у клиентов из категории безработных отсутствует доход
# для этого провожу индексацию данных по двум столбцам с условием одновременного == 'NaN'
print(data[(data['days_employed']==data['days_employed'].isnull()) & (data['total_income']==data['total_income'].isnull())])
print('В результате проверки получен пустой датафрейм, следовательно гипотеза не подтверждена. Необходимо заполнить пропуски данных.')

data['days_employed'] = data['days_employed'].fillna(0)
# заменяем пропуски данных в стобце 'days_employed' на 0, поскольку в дальнейшем данные из этого столбца нам не потребуются.
print('_________________________________________________')
print('Максимальное значение уровня значение уровня дохода: ', data['total_income'].max())
print('Минимальное значение уровня значение уровня дохода: ', data['total_income'].min())
print('Среднее значение уровня значение уровня дохода: ', data['total_income'].mean())
print('Медианное значение уровня значение уровня дохода: ', data['total_income'].median())
print('Заполняю пропуски в столбце total_income медианным значением дохода')
data['total_income'] = data['total_income'].fillna(data['total_income'].median())
print('_________________________________________________')
#Получаю обновленную информацию по датафрейму
data.info()

Empty DataFrame
Columns: [children, days_employed, dob_years, education, education_id, family_status, family_status_id, gender, income_type, debt, total_income, purpose]
Index: []
В результате проверки получен пустой датафрейм, следовательно гипотеза не подтверждена. Необходимо заполнить пропуски данных.
_________________________________________________
Максимальное значение уровня значение уровня дохода:  2265604.028722744
Минимальное значение уровня значение уровня дохода:  20667.26379327158
Среднее значение уровня значение уровня дохода:  167422.30220817294
Медианное значение уровня значение уровня дохода:  145017.93753253992
Заполняю пропуски в столбце total_income медианным значением дохода
_________________________________________________
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education  

### Вывод

В датафрейме обнаружены пропущенные значения типа 'NaN' в дух столбцах 'days_employed' и 'total_income'. 
Пропуски данных из столбца 'days_employed'я заполнил значением 0, поскольку в дальнейшем данные из этого столбца нам не потребуются.
Пропуски данных из столбца 'total_income'я заолнил медианным значением уровня заработной платы, поскольку это наиболее объективный метод в данном случае.
К наиболее вероятным причинам возникновения пропусков в данных столбцах относятся проблемы при преобразовании данных.

### Обработка артефактов

In [50]:
#Обработаю артефакты в толбце 'children'
data['children'] = data['children'].replace(20, 2) #методом replace() заменяю значение 20 на 2
data['children'] = data['children'].replace(-1, 1) #методом replace() заменяю значение -1 на 1
print(data['children'].value_counts()) #проверяю получившиеся значения в столбце 'children'

# умножая на '-1' перевожу отрицательные значения в столбце 'days_employed' в положительные
data.loc[data['days_employed'] < 0,'days_employed'] = data['days_employed'] * (-1)

if data[data['days_employed'] < 0]['days_employed'].count() > 0: #Проверяю получившиеся значения в столбце 'days_employed'
    print('В столбце "days_employed" остались отрицательные значения!')
else:
    print('Отрицательных значений в столбце "days_employed": ', data[data['days_employed'] < 0]['days_employed'].count())
    print('Eaaaah! В столбце "days_employed" все очень положительно!')

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64
Отрицательных значений в столбце "days_employed":  0
Eaaaah! В столбце "days_employed" все очень положительно!


### Вывод

Найденные в столбцах 'children' и 'days_employed' артефакты заменены на объективные значения.

### Замена типа данных

In [51]:
#заменяю типы данных в столбцах 'days_employed' и 'total_income' с вещественного на целочисленый методом .astype('int')
data['days_employed'] = data['days_employed'].astype('int')
data['total_income'] = data['total_income'].astype('int')
data.info()
#Проверяю результат на 5 первых значениях столбцов 'days_employed' и 'total_income'
print(data[['days_employed', 'total_income']].head(5)) 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB
   days_employed  total_income
0           8437        253875
1           4024        112080
2           5623        145885
3           4124        267628
4         340266        158616


### Вывод

Для более наглядного предоставления данных заменяю типы данных в столбцах 'days_employed' и 'total_income' с вещественного на целочисленый методом .astype('int')

### Обработка дубликатов

In [52]:
#В столбцах 'family_status' и 'education' имеются дубликаты одних и тех же данных записаных в разном регистре.
data['family_status'] = data['family_status'].str.lower() 
data['education'] = data['education'].str.lower() #Привожу все данные к одному виду - нижнему регистру.
print('Уникальные значения в столбце "образование": ', data['family_status'].unique())
print('Уникальные значения в столбце "образование": ', data['education'].unique())
print('Количество дубликатов в таблице:', data.duplicated().sum()) #Выявляю дублирующиеся строки
data = data.drop_duplicates().reset_index(drop=True) #Удаляю дубликаты методом .drop_duplicates().reset_index()
print('Количество дубликатов в таблице после удаления:', data.duplicated().sum())

Уникальные значения в столбце "образование":  ['женат / замужем' 'гражданский брак' 'вдовец / вдова' 'в разводе'
 'не женат / не замужем']
Уникальные значения в столбце "образование":  ['высшее' 'среднее' 'неоконченное высшее' 'начальное' 'ученая степень']
Количество дубликатов в таблице: 71
Количество дубликатов в таблице после удаления: 0


### Вывод

Дублирующиеся значения приведены к общему виду, применением метода .str.lower(). После этого выявлены и удалены дублирующиеся строки. Возможные причины возникновения дубликатов в данном случае: человеческий фактор, ошибки при конвертировании датафрейма.

### Лемматизация

In [53]:
from pymystem3 import Mystem #Импортирую библиотеку с функцией лемматизации
from collections import Counter #Импортирую библиотеку с функцией счетчика
m = Mystem()

def lemmatize_data(row): #Объявляю функцию для лемматизации строк
    lemmas = m.lemmatize(row)
    return lemmas

# Методом .apply() применяю функцию лемматизации к столбцу 'purpose' датафрейма, добавляю столбец 'purpose_lemmas'
data['purpose'] = data['purpose'].apply(lemmatize_data) 
#Суммирую все значения столбца 'purpose_lemmas' и применяю к ним счетчик Counter()
print('Словарь лемм столбца "purpose": ', Counter(data['purpose'].sum())) 


Словарь лемм столбца "purpose":  Counter({' ': 33570, '\n': 21454, 'недвижимость': 6351, 'покупка': 5897, 'жилье': 4460, 'автомобиль': 4306, 'образование': 4013, 'с': 2918, 'операция': 2604, 'свадьба': 2324, 'свой': 2230, 'на': 2222, 'строительство': 1878, 'высокий': 1374, 'получение': 1314, 'коммерческий': 1311, 'для': 1289, 'жилой': 1230, 'сделка': 941, 'дополнительный': 906, 'заниматься': 904, 'проведение': 768, 'сыграть': 765, 'сдача': 651, 'семья': 638, 'собственный': 635, 'со': 627, 'ремонт': 607, 'подержанный': 486, 'подержать': 478, 'приобретение': 461, 'профильный': 436})


### Вывод

С помощью лемматизации столбца 'purpose' определены основные категории целей получения кредита, а именно -  'ремонт', 'образование', 'свадьба', 'автомобиль', 'жилье', 'недвижимость', 'строительство'

### Категоризация данных

In [54]:
#Для категоризации данных из столбца 'дети' необходимо определить две группы клиентов: 'нет детей', 'есть дети'
def children_categorize(row): #Объявляю функцию для определения категорий
    if row == 0:
        return 'нет детей'
    return 'есть дети'
#Применяю функцию для определения категорий, добавляю в таблицу столбец 'children_category'
data['children_category'] = data['children'].apply(children_categorize)
print('Категории по наличию детей')
print(data['children_category'].value_counts()) #Проверяю работу функции для определения категорий
print()
#Для категоризации данных из столбца 'семейное положение' необходимо составить словарь с идентификаторами
family = data[['family_status','family_status_id']] #Выделяю интересующие столбцы в отдельную таблицу
family = family.drop_duplicates().reset_index(drop=True) #Удаляю дубликаты, сбрасываю индексы
print('Категории данных по семейному положению')
print(family)
print()
#Для категоризации данных из столбца 'ежемесячный доход', определяю 4 уровня дохода: низкий, средний, высокий, очень высокий
quant = data['total_income'].quantile([.25, .5, .75]) #Объявляю переменную, применяю к столбцу 'total_income' метод .quantile()
def income(income_level): #Объявляю функцию для определения категорий
    if income_level <= quant[0.25]:
        return 'низкий уровень дохода'
    if income_level <= quant[0.50]:
        return 'средний уровень дохода'
    if income_level <= quant[0.75]:
        return 'высокий уровень дохода'
    return 'очень высокий уровень дохода'
print()
#Применяю функцию для определения категорий, добавляю в таблицу столбец 'income_group'
data['income_group'] = data['total_income'].apply(income)
print('Категории по ежемесячному доходу')
print(data['income_group'].value_counts()) #Проверяю работу функции для определения категорий
print()
#Для категоризации данных из списка 'цель получения кредита', выбираю нужные леммы и собираю из них список
lemmas_list = ['ремонт', 'строительство', 'образование', 'свадьба', 'автомобиль', 'жилье', 'недвижимость']

def purpose_categorize(row):
    #lemmas = m.lemmatize(row)
    for i in lemmas_list:
        if i in row:
            return i
#Применяю функцию для определения категорий, заменяю данные в столбце 'purpose'
data['purpose'] = data['purpose'].apply(purpose_categorize)
print('Категории по цели получения кредита')
print(data['purpose'].value_counts())

Категории по наличию детей
нет детей    14091
есть дети     7363
Name: children_category, dtype: int64

Категории данных по семейному положению
           family_status  family_status_id
0        женат / замужем                 0
1       гражданский брак                 1
2         вдовец / вдова                 2
3              в разводе                 3
4  не женат / не замужем                 4


Категории по ежемесячному доходу
средний уровень дохода          6415
очень высокий уровень дохода    5364
низкий уровень дохода           5364
высокий уровень дохода          4311
Name: income_group, dtype: int64

Категории по цели получения кредита
недвижимость     4473
автомобиль       4306
образование      4013
жилье            3853
свадьба          2324
строительство    1878
ремонт            607
Name: purpose, dtype: int64


### Вывод

Данные, необходимые для ответов на вопросы категорированы. Выделен "словарь" данных по семейному положению для дальнейшего применения в сводных таблицах.

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [55]:
#Ответ на поставленный вопрос можно получить двумя способами
#Перый способ составить - сводную таблицу, по интересующим нас данным
children_pivot = data.pivot_table(index = ['children_category'], values = 'debt')
print(children_pivot.sort_values(by = 'debt', ascending=False))

print('____________________________________')
#Второй способ - сгруппировать нужные данные в новый датасет
child_dependence = pd.DataFrame(data = data.groupby('children_category').agg({'debt':['count','sum']}))
new_names = ['кол-во клиентов','клиенты имевшие задолженность']
child_dependence.set_axis(new_names, axis = 'columns', inplace = True)
#Вероятность возникновения задолженности отобразить в новом столбце
child_dependence['доля должников'] = child_dependence['клиенты имевшие задолженность'] / child_dependence['кол-во клиентов'] * 100
print(child_dependence.sort_values(by = 'доля должников', ascending=False))

                       debt
children_category          
есть дети          0.092082
нет детей          0.075438
____________________________________
                   кол-во клиентов  клиенты имевшие задолженность  \
children_category                                                   
есть дети                     7363                            678   
нет детей                    14091                           1063   

                   доля должников  
children_category                  
есть дети                9.208203  
нет детей                7.543822  


### Вывод

9,2% семей с детьми имеют проблемы с возвратом кредита в срок
7,5% семей без детей имеют проблемы с возвратом кредита в срок

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [56]:
#Ответ на поставленный вопрос можно получить двумя способами
#Перый способ составить - сводную таблицу, по интересующим нас данным
family_pivot = data.pivot_table(index = ['family_status'], values = 'debt')
print(family_pivot.sort_values(by = 'debt', ascending=False))

print('____________________________________')
#Второй способ - сгруппировать нужные данные в новый датасет
family_dependence = pd.DataFrame(data = data.groupby('family_status').agg({'debt':['count','sum']}))
family_dependence.set_axis(new_names, axis = 'columns', inplace = True)
#Вероятность возникновения задолженности отобразить в новом столбце
family_dependence['доля должников'] = family_dependence['клиенты имевшие задолженность'] / family_dependence['кол-во клиентов'] * 100
print(family_dependence.sort_values(by = 'доля должников', ascending=False))

                           debt
family_status                  
не женат / не замужем  0.097509
гражданский брак       0.093471
женат / замужем        0.075452
в разводе              0.071130
вдовец / вдова         0.065693
____________________________________
                       кол-во клиентов  клиенты имевшие задолженность  \
family_status                                                           
не женат / не замужем             2810                            274   
гражданский брак                  4151                            388   
женат / замужем                  12339                            931   
в разводе                         1195                             85   
вдовец / вдова                     959                             63   

                       доля должников  
family_status                          
не женат / не замужем        9.750890  
гражданский брак             9.347145  
женат / замужем              7.545182  
в разводе                  

### Вывод

Получено распределение должников по категриям семейного положения. Наиболее часто задолженности по кредитам возникают в категории 'не женат / не замужем', наиболее редко задолженности по кредитам возникают в категории 'вдовец / вдова'.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [57]:
#Ответ на поставленный вопрос можно получить двумя способами
#Перый способ составить - сводную таблицу, по интересующим нас данным
income_pivot = data.pivot_table(index = ['income_group'], values = 'debt')
print(income_pivot.sort_values(by = 'debt', ascending=False))

print('____________________________________')
#Второй способ - сгруппировать нужные данные в новый датасет
income_dependence = pd.DataFrame(data = data.groupby('income_group').agg({'debt':['count','sum']}))
income_dependence.set_axis(new_names, axis = 'columns', inplace = True)
#Вероятность возникновения задолженности отобразить в новом столбце
income_dependence['доля должников'] = income_dependence['клиенты имевшие задолженность'] / income_dependence['кол-во клиентов'] * 100
print(income_dependence.sort_values(by = 'доля должников', ascending=False))

                                  debt
income_group                          
высокий уровень дохода        0.089074
средний уровень дохода        0.085269
низкий уровень дохода         0.079605
очень высокий уровень дохода  0.071402
____________________________________
                              кол-во клиентов  клиенты имевшие задолженность  \
income_group                                                                   
высокий уровень дохода                   4311                            384   
средний уровень дохода                   6415                            547   
низкий уровень дохода                    5364                            427   
очень высокий уровень дохода             5364                            383   

                              доля должников  
income_group                                  
высокий уровень дохода              8.907446  
средний уровень дохода              8.526890  
низкий уровень дохода               7.960477  
очень высокий

### Вывод

Получено распределение должников по категриям ежемесячного дохода. Наиболее часто задолженности по кредитам возникают в категории 'высокий уровень дохода', наиболее редко задолженности по кредитам возникают в категории 'очень высокий уровень дохода'.

- Как разные цели кредита влияют на его возврат в срок?

In [58]:
#Ответ на поставленный вопрос можно получить двумя способами
#Перый способ составить - сводную таблицу, по интересующим нас данным
purpose_pivot = data.pivot_table(index = ['purpose'], values = 'debt')
print(purpose_pivot.sort_values(by = 'debt', ascending=False))

print('____________________________________')
#Второй способ - сгруппировать нужные данные в новый датасет
purpose_dependence = pd.DataFrame(data = data.groupby('purpose').agg({'debt':['count','sum']}))
purpose_dependence.set_axis(new_names, axis = 'columns', inplace = True)
#Вероятность возникновения задолженности отобразить в новом столбце
purpose_dependence['доля должников'] = purpose_dependence['клиенты имевшие задолженность'] / purpose_dependence['кол-во клиентов'] * 100
print(purpose_dependence.sort_values(by = 'доля должников', ascending=False))

                   debt
purpose                
автомобиль     0.093590
образование    0.092200
свадьба        0.080034
строительство  0.076677
недвижимость   0.073776
жилье          0.070854
ремонт         0.057661
____________________________________
               кол-во клиентов  клиенты имевшие задолженность  доля должников
purpose                                                                      
автомобиль                4306                            403        9.359034
образование               4013                            370        9.220035
свадьба                   2324                            186        8.003442
строительство             1878                            144        7.667732
недвижимость              4473                            330        7.377599
жилье                     3853                            273        7.085388
ремонт                     607                             35        5.766063


### Вывод

Получено распределение должников по категриям целей получения кредитов. Наиболее часто задолженности по кредитам возникают в категории 'автомобиль', наиболее редко задолженности по кредитам возникают в категории 'ремонт'.

### Шаг 4. Общий вывод

Проведенное исследование надежности заемщиков позволило не только выявить наиболее и наименее проблемные группы заемщиков в различных категорях, но и дало возможность составления своего рода рейтингов заемщиков исходя из их принадлежности к той или иной категори. Например, наиболее привлекательным заемщиком будет вдовец/вдова с очень высоким уровнем дохода, не имеющий детей, взявший кредит на ремонт.